# Generating Inputs for the ABFE Benchmark Burn-in Set

This notebook generates parameters for selected systems for the "burn-in" set. 

The workflow starts with generating OpenMM systems for the protien and ligands. Parameters for the ligands are derrived from the "Sage" force field with AM1-BCC charges, while the protein using AMBER14SB and the TIP3P model is used for the water molecules. 

These openMM systems are converted for use in the ParmEd library, before being exported as GROMACS gro/top input files.

The burn-in set contains the following systems:
- BRD4(1) with 2 ligands
- cmet with 2 ligands
- CycloD with 2 frgaments and a merged ligand
- jnk1 with 2 ligands
- p38 with 2 ligands
- thrombin with 2 ligands

This notebook was based off the super notebook examples that are available on the Open Force Field GitHub page.

### Imports

In [1]:
import os
import shutil
import numpy as np
import subprocess
from subprocess import Popen, PIPE
from pprint import pprint

import openmm
from openmm import unit
from openmm import app
from openmm.app import PME, HBonds, PDBFile, Modeller

from rdkit import Chem
from rdkit.Chem import AllChem

import parmed as pmd

import openff.toolkit
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.tests.utils import (
    compare_system_energies,
    compare_system_parameters)
from openff.toolkit.typing.engines.smirnoff import ForceField

from openmmforcefields.generators import SMIRNOFFTemplateGenerator

In [2]:
def copy_over_ligand_files(input_sdf_filename, output_sdf_filename):
    """
    Copies an SD File to the desired location, also renaming the compound to be LIG
    """
    
    # read in the first compound in the SD File
    suppl = Chem.SDMolSupplier(input_sdf_filename)
    mol = [x for x in suppl][0]
    
    # rename the molecule
    mol.SetProp("_Name", "LIG")
    
    # write out file in to the specified location
    with Chem.SDWriter(output_sdf_filename) as w:
        w.write(mol)
            

#copy_over_ligand_files("./structures/brd4/ligand1.sdf", 
#                       "./parameters/brd4/ligand1/ligand1.sdf")            

### Generate parameters for each system:

In [3]:
# List of protein names, for the systems that will be set up:
protein_list = ['thrombin',
                'cmet',
                'cyclod',
                'jnk1',
                'p38',
                'brd4',
               ]

In [11]:
# List of proteins we need to set up:
protein_list = ['thrombin',
                'cmet',
                'cyclod',
                'jnk1',
                'p38',
                'brd4',
               ]

# Loop through each system, by protein
for p in protein_list:
    
    print(f"Starting on {p}...")
    
    # make a folder for parameters
    if not os.path.exists(f'./parameters/{p}'):
        os.makedirs(f'./parameters/{p}')

    # Generate protein parameters
    protein_pdb_path = f'./structures/{p}/protein.pdb'
    protein_pdb = openmm.app.PDBFile(protein_pdb_path)
    
    # Generate water parameters
    if os.path.isfile(f'./structures/{p}/water.pdb'):
        water_pdb_path = f'./structures/{p}/water.pdb'
        water_pdb = openmm.app.PDBFile(water_pdb_path)
    else:
        water_pdb_path = False
    
    # get ligand filenames:
    if p == 'cyclod':
        ligand_filenames = ['ligand1', 'ligand2', 'ligand3']
    else:
        ligand_filenames = ['ligand1', 'ligand2']

    # generate files for each ligand
    for l in ligand_filenames:
        
        print(f"  Working on {l}...")
        
        # directory for each ligand
        if not os.path.exists(f'./parameters/{p}/{l}'):
            os.makedirs(f'./parameters/{p}/{l}')
        
        # Reset the force field object
        omm_forcefield = openmm.app.ForceField("amber99sbildn.xml", "tip3p.xml")
        
        # Generate sage parameters
        ligand_sdf_path = f'./structures/{p}/{l}.sdf'
        ligand = Molecule.from_file(ligand_sdf_path)
        ligand_positions = ligand.conformers[0]
        ligand_topology = ligand.to_topology()
        
        sage = SMIRNOFFTemplateGenerator(
            forcefield="openff-2.0.0.offxml", molecules=[ligand]
        )
        omm_forcefield.registerTemplateGenerator(sage.generator)
        
        # Build the ligand only system
        lig_modeller = openmm.app.Modeller(
            ligand_topology.to_openmm(),
            ligand_positions
        )
        lig_modeller.addSolvent(
            omm_forcefield,
            model="tip3p",
            padding=1.2 * unit.nanometer,
            ionicStrength=0.15 * unit.molar,
        )
        lig_system = omm_forcefield.createSystem(
            lig_modeller.topology,
            nonbondedMethod=openmm.app.PME,
            nonbondedCutoff=9 * unit.angstrom,
            constraints=openmm.app.HBonds,
            rigidWater=True,
        )
        
        solvated_lig_topology = lig_modeller.getTopology()
        solvated_lig_positions = lig_modeller.getPositions()
        
        # Export to GROMACS via ParmEd's 
        # Note: ParmEd's GROMACS exporter can't handle constraints from openmm, 
        # so we need a variant for export without them
        lig_export_system = omm_forcefield.createSystem(
            lig_modeller.topology,
            nonbondedMethod=openmm.app.PME,
            constraints=None,
            rigidWater=False,
        )
        
        # Combine the topology, system and positions into a ParmEd Structure
        pmd_ligand_struct = pmd.openmm.load_topology(
            solvated_lig_topology, lig_export_system, solvated_lig_positions)

        # Export solvated ligand GROMACS/AMBER files.
        pmd_ligand_struct.save(f'./parameters/{p}/{l}/ligand.top', overwrite=True)
        pmd_ligand_struct.save(f'./parameters/{p}/{l}/ligand.gro', overwrite=True)
        pmd_ligand_struct.save(f'./parameters/{p}/{l}/ligand.prmtop', overwrite=True)
        pmd_ligand_struct.save(f'./parameters/{p}/{l}/ligand.inpcrd', overwrite=True)
   
        # Build the complex:
        # Order: protein, ligand and then any waters
        # Load the protein strucutre into modeller
        complex_modeller = openmm.app.Modeller(
            protein_pdb.topology, protein_pdb.positions)
        
        # Add the ligand
        # Under the hood, this line uses the OpenFF Toolkit to generate new parameters for the ligand
        complex_modeller.add(
            ligand_topology.to_openmm(), ligand_positions)
        
        if os.path.isfile(f'./structures/{p}/water.pdb'):
            complex_modeller.add(water_pdb.topology, water_pdb.positions)

        # solvate it in 0.15 M NaCl solution
        complex_modeller.addSolvent(
            omm_forcefield,
            model="tip3p",
            padding=1.2 * unit.nanometer,
            ionicStrength=0.15 * unit.molar,
        )
        
        # Construct the OpenMM System, which stores the forces for the system
        complex_system = omm_forcefield.createSystem(
            complex_modeller.topology,
            nonbondedMethod=openmm.app.PME,
            nonbondedCutoff=9 * unit.angstrom,
            constraints=openmm.app.HBonds,
            rigidWater=True,
        )
        complex_topology = complex_modeller.getTopology()
        complex_positions = complex_modeller.getPositions()
        
        # Output using GROMACS:
        # ParmEd's GROMACS exporter can't handle constraints from openmm, so we need a variant for export without them
        complex_export_system = omm_forcefield.createSystem(
            complex_modeller.topology,
            nonbondedMethod=openmm.app.PME,
            constraints=None,
            rigidWater=False,
        )

        # Combine the topology, system and positions into a ParmEd Structure
        pmd_complex_struct = pmd.openmm.load_topology(
            complex_topology, 
            complex_export_system, 
            complex_positions)

        # Export GROMACS files.
        pmd_complex_struct.save(f'./parameters/{p}/{l}/complex.top', overwrite=True)
        pmd_complex_struct.save(f'./parameters/{p}/{l}/complex.gro', overwrite=True)
        # Write the protein structure to AMBER format.
        pmd_complex_struct.save(f'./parameters/{p}/{l}/complex.prmtop', overwrite=True)
        pmd_complex_struct.save(f'./parameters/{p}/{l}/complex.inpcrd', overwrite=True)

        # Check the models worked by reading them back in:
        amber_structure = pmd.load_file(f'./parameters/{p}/{l}/complex.prmtop',
                                           f'./parameters/{p}/{l}/complex.inpcrd')
        
        # Convert the Structure to an OpenMM System. Note that by
        # default ParmEd will add a CMMotionRemover force to the
        # System, and won't constrain the hydrogen bonds.
        amber_system = amber_structure.createSystem(
            nonbondedMethod=PME,
            nonbondedCutoff=9.0 * unit.angstrom,
            switchDistance=0.0 * unit.angstrom,
            constraints=HBonds,
            removeCMMotion=False,
        )
        
        gmx_structure = pmd.load_file(f'./parameters/{p}/{l}/complex.gro',
                                           f'./parameters/{p}/{l}/complex.top')
        
        gmx_system = gmx_structure.createSystem(
            nonbondedMethod=openmm.app.PME,
            nonbondedCutoff=9.0 * unit.angstrom,
            switchDistance=0.0 * unit.angstrom,
            constraints=openmm.app.HBonds,
            rigidWater=True,
        )
        
        amber_energies, omm_energies = compare_system_energies(
            amber_system,
            complex_export_system,
            #complex_system,
            amber_structure.positions,
            amber_structure.box_vectors,
            rtol=1e-3,
        )
        
        print("\nOriginal OpenMM System:")
        print("-----------------------")
        pprint(omm_energies)
        
        print("System loaded from AMBER files:")
        print("-------------------------------")
        pprint(amber_energies)
        
        

Starting on thrombin...
  Working on ligand1...

Original OpenMM System:
-----------------------
{'HarmonicAngleForce': Quantity(value=7302.94775390625, unit=kilojoule/mole),
 'HarmonicBondForce': Quantity(value=194631.375, unit=kilojoule/mole),
 'NonbondedForce': Quantity(value=190583.6374440454, unit=kilojoule/mole),
 'PeriodicTorsionForce': Quantity(value=12374.908203125, unit=kilojoule/mole)}
System loaded from AMBER files:
-------------------------------
{'HarmonicAngleForce': Quantity(value=7302.9482421875, unit=kilojoule/mole),
 'HarmonicBondForce': Quantity(value=194631.375, unit=kilojoule/mole),
 'NonbondedForce': Quantity(value=190580.99498227285, unit=kilojoule/mole),
 'PeriodicTorsionForce': Quantity(value=12374.9091796875, unit=kilojoule/mole)}
  Working on ligand2...

Original OpenMM System:
-----------------------
{'HarmonicAngleForce': Quantity(value=7320.12646484375, unit=kilojoule/mole),
 'HarmonicBondForce': Quantity(value=194633.265625, unit=kilojoule/mole),
 'Nonbo

<b>NOTE:</b> This function should not be rerun by participants of the burn in set. This is to ensure that everyone is using the same parameters. If you need to play with the ParmEd system further, please use the example below to read in the AMBER inpt files.